In [128]:
#importing pandas library and loading Data.
import pandas as pd
dataFile = r'G:\documents\books rating\FD2.csv'
data = pd.read_csv(dataFile,header=0,index_col =0,encoding = 'unicode_escape',error_bad_lines=False)

In [129]:
data.head()

,Name,Specialization,EXP,Rating
FID,,,,
12,AS,ASST.PRO,5,4
13,HG,ASST.PRO,4,5
14,PI,HOD,1,2
15,HY,PROF,9,3
16,KI,HOD,7,4


In [130]:
#Loading Specialization
Spe = r'G:\documents\books rating\Specialization.csv'
Specialization = pd.read_csv(Spe,header=0,index_col =0,error_bad_lines = False,encoding='latin-1')

In [131]:
Specialization.head()

,Spec,Last Passed College,Name,Designation
FID,,,,
12,IOT,NIT(rkl),AS,ASST.PRO
13,AI/ML,IIT(kgp),HG,ASST.PRO
14,Web,ITER,PI,HOD
15,AI/ML,IIT(bombay),HY,PROF
16,Web,IIIT(Hy),KI,HOD


In [132]:
#Function to get the a Spec,LPC,name,Desig for a given FID
def Speci(FID):
    Spec=Specialization.get_value(FID,'Spec')
    LPC = Specialization.get_value(FID,'Last Passed College')
    name=Specialization.get_value(FID,'Name')
    Desig=Specialization.get_value(FID,'Designation')
    return Spec,LPC,name,Desig
Speci(12)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] access

('IOT', 'NIT(rkl)', 'AS', 'ASST.PRO')

In [133]:
#Function to get N favorite Specialization of an User in a sorted manner based upon ratings.
def FavSub(Name,N):
    userRatings = data[data['Name'] == 'Name']
    SortedRatings = pd.DataFrame.sort_values(userRatings,['Rating'],ascending = [0])[:N]
    SortedRatings['Spec'] = SortedRatings['Name'].apply(Speci)
    return SortedRatings

In [134]:
FavSub('AS',5)

,Name,Specialization,EXP,Rating,Spec
FID,,,,,


In [135]:
data.shape

(5, 4)

In [136]:
#Creating the rating matrix.
userItemRatingMatrix=pd.pivot_table(data, values='Rating',
                                    index=['Name'], columns=['FID'])

In [137]:
userItemRatingMatrix.head(10)

FID,12,13,14,15,16
Name,,,,,
AS,4.0,NaN,NaN,NaN,NaN
HG,NaN,5.0,NaN,NaN,NaN
HY,NaN,NaN,NaN,3.0,NaN
KI,NaN,NaN,NaN,NaN,4.0
PI,NaN,NaN,2.0,NaN,NaN


In [138]:
user1 = 'AS'
user2 = 'KI'

In [139]:
#Accessing the user1 and user2 rating values.
user1Ratings = userItemRatingMatrix.transpose()[user1]
user2Ratings = userItemRatingMatrix.transpose()[user2]

In [140]:
user1Ratings.head()

FID
12    4.0
13    NaN
14    NaN
15    NaN
16    NaN
Name: AS, dtype: float64

In [141]:
#Importing in-built hamming funct from scipy lib. 
from scipy.spatial.distance import hamming
hamming(user1,user2)

1.0

In [142]:
#Calculating the proximity between the 2 users.
import numpy as np
def distance(user1,user2):
    try:
        user1Ratings = userItemRatingMatrix.transpose()[user1]
        user2Ratings = userItemRatingMatrix.transpose()[user2]
        
        distance = hamming(user1,user2)
    except:
        distance = np.NaN
    return distance

In [143]:
distance('AS','KI')

1.0

In [144]:
#Creating a dataFrame having all the User id's. 
Name = 'AS'
allUser  = pd.DataFrame(userItemRatingMatrix.index)


In [145]:
#Removing active user from allUser.
allUser = allUser[allUser.Name!=Name]

In [146]:
allUser.head()

,Name
1,HG
2,HY
3,KI
4,PI


In [147]:
#Adding the Distance column to allUser by applying distance funct.
allUser['distance'] = allUser['Name'].apply(lambda x:distance(Name,x))

In [148]:
allUser.head(10)

,Name,distance
1,HG,1.0
2,HY,1.0
3,KI,1.0
4,PI,1.0


In [149]:
def nearestNeighbors(Name,K=10):
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.Name!='Name']
    allUsers["distance"] = allUsers["Name"].apply(lambda x: distance(Name,x))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["Name"][:K]
    return KnearestUsers

In [150]:
KnearestUsers = nearestNeighbors(Name)

In [151]:
#Taking only those ratings of user which are present in KnearestUsers
NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]

In [152]:
NNRatings.head()

FID,12,13,14,15,16
Name,,,,,
AS,4.0,NaN,NaN,NaN,NaN
HG,NaN,5.0,NaN,NaN,NaN
HY,NaN,NaN,NaN,3.0,NaN
KI,NaN,NaN,NaN,NaN,4.0
PI,NaN,NaN,2.0,NaN,NaN


In [153]:
#applying mean over the previously found ratings
avgRating = NNRatings.apply(np.nanmean).dropna()

In [154]:
avgRating


FID
12    4.0
13    5.0
14    2.0
15    3.0
16    4.0
dtype: float64

In [155]:
#finding all the Domains which are being done by our ACTIVE USER.
DomainWorked = userItemRatingMatrix.transpose()[Name].dropna().index
DomainWorked

Int64Index([12], dtype='int64', name='FID')

In [156]:
avgRating = avgRating[~avgRating.index.isin(DomainWorked)]

In [157]:
#Arranging the N-Teacher in the descending order of their ratings 
N =3
topSpec = avgRating.sort_values(ascending = False).index[:N]

In [158]:
pd.Series(topSpec).apply(Speci)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] access

0    (AI/ML, IIT(kgp), HG, ASST.PRO)
1           (Web, IIIT(Hy), KI, HOD)
2     (AI/ML, IIT(bombay), HY, PROF)
Name: FID, dtype: object

In [159]:
#Arranging the N-Teacher in the descending order of their ratings using the given function
def topN(Name,N=5):
    KnearestUsers = nearestNeighbors(Name)
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers)]
    avgRating = NNRatings.apply(np.nanmean).dropna()
    DomainWorked = userItemRatingMatrix.transpose()[Name].dropna().index
    avgRating = avgRating[~avgRating.index.isin(DomainWorked)]
    topSpec = avgRating.sort_values(ascending=False).index[:N]
    return pd.Series(topSpec).apply(Speci)

In [160]:
FavSub('AS',5)

,Name,Specialization,EXP,Rating,Spec
FID,,,,,


In [162]:
topN('PI',5)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] access

0    (AI/ML, IIT(kgp), HG, ASST.PRO)
1           (Web, IIIT(Hy), KI, HOD)
2      (IOT, NIT(rkl), AS, ASST.PRO)
3     (AI/ML, IIT(bombay), HY, PROF)
Name: FID, dtype: object

Thank you!!!!